In [15]:
import pandas as pd
import requests
import time
from tqdm import tqdm
from collections import Counter


In [16]:
GITHUB_TOKEN = "INSERT_TOKEN"
headers = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3+json"
}

In [17]:
path_to_file = r'output_files\prs_reviews.csv'
closed_prs = pd.read_csv(path_to_file)

In [18]:
has_reviews = closed_prs[closed_prs['reviews_data'].str.len() > 2]
has_reviews

,id,number,user,user_id,agent,repo_url,html_url,reviews_url,reviews_data,review_counts_map
0,2438086945,88748,iamrajjoshi,33237075,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/88748,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'GabeVillalobos', 'state': 'COMMENTE...",{'GabeVillalobos': 2}
1,2265431531,83085,ArthurKnaus,7033940,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/83085,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'obostjancic', 'state': 'APPROVED', ...",{'obostjancic': 1}
2,2622011651,94465,bukzor,640328,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/94465,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'joshuarli', 'state': 'APPROVED', 'b...",{'joshuarli': 1}
3,2565399631,92785,dashed,139499,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/92785,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'isabellaenriquez', 'state': 'APPROV...",{'isabellaenriquez': 1}
4,2374801945,86438,brendanhsentry,171613822,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/86438,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'isabellaenriquez', 'state': 'APPROV...","{'isabellaenriquez': 1, 'dashed': 1}"
...,...,...,...,...,...,...,...,...,...,...
9037,3152227876,2121,devin-ai-integration[bot],158243242,Devin,https://api.github.com/repos/assistant-ui/assi...,https://github.com/assistant-ui/assistant-ui/p...,https://api.github.com/repos/assistant-ui/assi...,"[{'user': 'greptile-apps[bot]', 'state': 'COMM...",{'greptile-apps[bot]': 1}
9042,2876371914,54663,devin-ai-integration[bot],158243242,Devin,https://api.github.com/repos/airbytehq/airbyte,https://github.com/airbytehq/airbyte/pull/54663,https://api.github.com/repos/airbytehq/airbyte...,"[{'user': 'pnilan', 'state': 'APPROVED', 'body...",{'pnilan': 1}
9045,3275451449,2766,devin-ai-integration[bot],158243242,Devin,https://api.github.com/repos/liam-hq/liam,https://github.com/liam-hq/liam/pull/2766,https://api.github.com/repos/liam-hq/liam/pull...,"[{'user': 'MH4GF', 'state': 'COMMENTED', 'body...","{'MH4GF': 3, 'copilot-pull-request-reviewer[bo..."
9046,3260325787,1541,devin-ai-integration[bot],158243242,Devin,https://api.github.com/repos/reflex-dev/reflex...,https://github.com/reflex-dev/reflex-web/pull/...,https://api.github.com/repos/reflex-dev/reflex...,"[{'user': 'greptile-apps[bot]', 'state': 'COMM...","{'greptile-apps[bot]': 1, 'Lendemor': 1}"


In [19]:
# --- 1. Preparação das colunas ---
# Garantir que as colunas existam para não dar erro de atribuição depois
if 'closed_by_user' not in has_reviews.columns:
    has_reviews['closed_by_user'] = None
if 'closing_method' not in has_reviews.columns:
    has_reviews['closing_method'] = None

# Filtra apenas quem ainda não tem a informação (para economizar API se rodar de novo)
prs_para_processar = has_reviews[has_reviews['closed_by_user'].isna()].copy()

print(f"Processando {len(prs_para_processar)} PRs...")

Processando 3323 PRs...


C:\Users\Renato\AppData\Local\Temp\ipykernel_10880\1612956348.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  has_reviews['closed_by_user'] = None
C:\Users\Renato\AppData\Local\Temp\ipykernel_10880\1612956348.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  has_reviews['closing_method'] = None


In [20]:
# --- FUNÇÃO INTELIGENTE COM AUTO-SLEEP ---
def get_pr_closure_info(row):
    url_pr = f"{row['repo_url']}/pulls/{row['number']}"
    
    try:
        # TENTATIVA 1: Endpoint da PR
        response = requests.get(url_pr, headers=headers, timeout=10)
        
        # Se der Rate Limit (403), dorme e tenta de novo
        if response.status_code == 403 and 'X-RateLimit-Reset' in response.headers:
            reset_time = int(response.headers['X-RateLimit-Reset'])
            sleep_time = reset_time - time.time() + 10 # +10s de segurança
            if sleep_time > 0:
                print(f"\n⛔ Rate Limit atingido! Dormindo {sleep_time/60:.1f} min...")
                time.sleep(sleep_time)
                return get_pr_closure_info(row) # Tenta de novo (recursão)

        if response.status_code == 200:
            data = response.json()
            if data.get('merged_by'):
                return data['merged_by']['login'], 'Merged'
            if data.get('state') == 'closed' and data.get('closed_by'):
                if data.get('merged_at'): 
                    return data['closed_by']['login'], 'Merged'
                return data['closed_by']['login'], 'Closed'

        # TENTATIVA 2: Fallback para Timeline
        url_events = f"{row['repo_url']}/issues/{row['number']}/events"
        resp_events = requests.get(url_events, headers=headers, timeout=10)
        
        # Rate Limit na Timeline
        if resp_events.status_code == 403 and 'X-RateLimit-Reset' in resp_events.headers:
            reset_time = int(resp_events.headers['X-RateLimit-Reset'])
            sleep_time = reset_time - time.time() + 10
            if sleep_time > 0:
                print(f"\n⛔ Rate Limit (Timeline)! Dormindo {sleep_time/60:.1f} min...")
                time.sleep(sleep_time)
                return get_pr_closure_info(row)
        
        if resp_events.status_code == 200:
            events = resp_events.json()
            # Procura Merge
            for event in events:
                if event['event'] == 'merged':
                    actor = event.get('actor')
                    return (actor.get('login') if actor else 'Ghost/Deleted'), 'Merged'
            # Procura Closed
            for event in events:
                if event['event'] == 'closed':
                    actor = event.get('actor')
                    return (actor.get('login') if actor else 'Ghost/Deleted'), 'Closed'

    except Exception as e:
        print(f"\nErro na PR {row.get('number')}: {e}")
        
    return None, None

# --- EXECUÇÃO DO LOOP ---
print("Iniciando processamento com Auto-Sleep ativado...")

# Filtra apenas o que falta processar
prs_para_processar = has_reviews[has_reviews['closed_by_user'].isna()]

for idx, row in tqdm(prs_para_processar.iterrows(), total=len(prs_para_processar)):
    user, method = get_pr_closure_info(row)
    
    if user:
        # Usa .loc para evitar o erro "must be a scalar"
        has_reviews.loc[idx, 'closed_by_user'] = user
        has_reviews.loc[idx, 'closing_method'] = method

print("\nFinalizado!")
has_reviews.head()

Iniciando processamento com Auto-Sleep ativado...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3323/3323 [1:00:42<00:00,  1.10s/it]


Finalizado!


,id,number,user,user_id,agent,repo_url,html_url,reviews_url,reviews_data,review_counts_map,closed_by_user,closing_method
0,2438086945,88748,iamrajjoshi,33237075,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/88748,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'GabeVillalobos', 'state': 'COMMENTE...",{'GabeVillalobos': 2},iamrajjoshi,Merged
1,2265431531,83085,ArthurKnaus,7033940,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/83085,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'obostjancic', 'state': 'APPROVED', ...",{'obostjancic': 1},ArthurKnaus,Merged
2,2622011651,94465,bukzor,640328,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/94465,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'joshuarli', 'state': 'APPROVED', 'b...",{'joshuarli': 1},bukzor,Merged
3,2565399631,92785,dashed,139499,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/92785,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'isabellaenriquez', 'state': 'APPROV...",{'isabellaenriquez': 1},dashed,Merged
4,2374801945,86438,brendanhsentry,171613822,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/86438,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'isabellaenriquez', 'state': 'APPROV...","{'isabellaenriquez': 1, 'dashed': 1}",brendanhsentry,Merged


In [21]:
has_reviews[has_reviews['closing_method'].isna()]

,id,number,user,user_id,agent,repo_url,html_url,reviews_url,reviews_data,review_counts_map,closed_by_user,closing_method
1845,3179981594,4265,Copilot,198982749,Copilot,https://api.github.com/repos/valkey-io/valkey-...,https://github.com/valkey-io/valkey-glide/pull...,https://api.github.com/repos/valkey-io/valkey-...,"[{'user': 'avifenesh', 'state': 'COMMENTED', '...",{'avifenesh': 3},None,None
1859,3078439427,62036,Copilot,198982749,Copilot,https://api.github.com/repos/dotnet/aspnetcore,https://github.com/dotnet/aspnetcore/pull/62036,https://api.github.com/repos/dotnet/aspnetcore...,"[{'user': 'captainsafia', 'state': 'COMMENTED'...","{'captainsafia': 8, 'copilot-swe-agent[bot]': ...",None,None
1912,3074609325,115732,Copilot,198982749,Copilot,https://api.github.com/repos/dotnet/runtime,https://github.com/dotnet/runtime/pull/115732,https://api.github.com/repos/dotnet/runtime/pu...,"[{'user': 'danmoseley', 'state': 'COMMENTED', ...","{'danmoseley': 1, 'copilot-swe-agent[bot]': 1}",None,None
1913,3074618957,115733,Copilot,198982749,Copilot,https://api.github.com/repos/dotnet/runtime,https://github.com/dotnet/runtime/pull/115733,https://api.github.com/repos/dotnet/runtime/pu...,"[{'user': 'segadora', 'state': 'COMMENTED', 'b...",{'segadora': 1},None,None
1930,3184463362,30291,Copilot,198982749,Copilot,https://api.github.com/repos/dotnet/maui,https://github.com/dotnet/maui/pull/30291,https://api.github.com/repos/dotnet/maui/pulls...,"[{'user': 'PureWeen', 'state': 'CHANGES_REQUES...","{'PureWeen': 5, 'copilot-swe-agent[bot]': 5}",None,None
2042,3169212086,1704,Copilot,198982749,Copilot,https://api.github.com/repos/microsoft/retina,https://github.com/microsoft/retina/pull/1704,https://api.github.com/repos/microsoft/retina/...,"[{'user': 'agrawaliti', 'state': 'DISMISSED', ...","{'agrawaliti': 2, 'SRodi': 3, 'copilot-swe-age...",None,None
2069,3146392650,35272,Copilot,198982749,Copilot,https://api.github.com/repos/Azure/azure-rest-...,https://github.com/Azure/azure-rest-api-specs/...,https://api.github.com/repos/Azure/azure-rest-...,"[{'user': 'mikeharder', 'state': 'COMMENTED', ...","{'mikeharder': 1, 'copilot-swe-agent[bot]': 1}",None,None
2165,3199881418,47088,Copilot,198982749,Copilot,https://api.github.com/repos/dotnet/docs,https://github.com/dotnet/docs/pull/47088,https://api.github.com/repos/dotnet/docs/pulls...,"[{'user': 'jkotas', 'state': 'COMMENTED', 'bod...","{'jkotas': 10, 'copilot-swe-agent[bot]': 10, '...",None,None
2248,3172846336,7729,Copilot,198982749,Copilot,https://api.github.com/repos/microsoft/typespec,https://github.com/microsoft/typespec/pull/7729,https://api.github.com/repos/microsoft/typespe...,"[{'user': 'JoshLove-msft', 'state': 'COMMENTED...","{'JoshLove-msft': 7, 'copilot-swe-agent[bot]': 6}",None,None
2258,3075857183,115762,Copilot,198982749,Copilot,https://api.github.com/repos/dotnet/runtime,https://github.com/dotnet/runtime/pull/115762,https://api.github.com/repos/dotnet/runtime/pu...,"[{'user': 'tarekgh', 'state': 'COMMENTED', 'bo...","{'tarekgh': 4, 'copilot-swe-agent[bot]': 8, 'j...",None,None


In [22]:
# Verifica se o valor é uma string E se não contém '[bot]'
has_reviews['has_human_closing_user'] = has_reviews['closed_by_user'].apply(
    lambda x: isinstance(x, str) and '[bot]' not in x
)

# Exibir o resultado
has_reviews

C:\Users\Renato\AppData\Local\Temp\ipykernel_10880\3156863303.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  has_reviews['has_human_closing_user'] = has_reviews['closed_by_user'].apply(


,id,number,user,user_id,agent,repo_url,html_url,reviews_url,reviews_data,review_counts_map,closed_by_user,closing_method,has_human_closing_user
0,2438086945,88748,iamrajjoshi,33237075,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/88748,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'GabeVillalobos', 'state': 'COMMENTE...",{'GabeVillalobos': 2},iamrajjoshi,Merged,True
1,2265431531,83085,ArthurKnaus,7033940,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/83085,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'obostjancic', 'state': 'APPROVED', ...",{'obostjancic': 1},ArthurKnaus,Merged,True
2,2622011651,94465,bukzor,640328,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/94465,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'joshuarli', 'state': 'APPROVED', 'b...",{'joshuarli': 1},bukzor,Merged,True
3,2565399631,92785,dashed,139499,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/92785,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'isabellaenriquez', 'state': 'APPROV...",{'isabellaenriquez': 1},dashed,Merged,True
4,2374801945,86438,brendanhsentry,171613822,Human,https://api.github.com/repos/getsentry/sentry,https://github.com/getsentry/sentry/pull/86438,https://api.github.com/repos/getsentry/sentry/...,"[{'user': 'isabellaenriquez', 'state': 'APPROV...","{'isabellaenriquez': 1, 'dashed': 1}",brendanhsentry,Merged,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9037,3152227876,2121,devin-ai-integration[bot],158243242,Devin,https://api.github.com/repos/assistant-ui/assi...,https://github.com/assistant-ui/assistant-ui/p...,https://api.github.com/repos/assistant-ui/assi...,"[{'user': 'greptile-apps[bot]', 'state': 'COMM...",{'greptile-apps[bot]': 1},Yonom,Closed,True
9042,2876371914,54663,devin-ai-integration[bot],158243242,Devin,https://api.github.com/repos/airbytehq/airbyte,https://github.com/airbytehq/airbyte/pull/54663,https://api.github.com/repos/airbytehq/airbyte...,"[{'user': 'pnilan', 'state': 'APPROVED', 'body...",{'pnilan': 1},pnilan,Merged,True
9045,3275451449,2766,devin-ai-integration[bot],158243242,Devin,https://api.github.com/repos/liam-hq/liam,https://github.com/liam-hq/liam/pull/2766,https://api.github.com/repos/liam-hq/liam/pull...,"[{'user': 'MH4GF', 'state': 'COMMENTED', 'body...","{'MH4GF': 3, 'copilot-pull-request-reviewer[bo...",NoritakaIkeda,Merged,True
9046,3260325787,1541,devin-ai-integration[bot],158243242,Devin,https://api.github.com/repos/reflex-dev/reflex...,https://github.com/reflex-dev/reflex-web/pull/...,https://api.github.com/repos/reflex-dev/reflex...,"[{'user': 'greptile-apps[bot]', 'state': 'COMM...","{'greptile-apps[bot]': 1, 'Lendemor': 1}",adhami3310,Merged,True


In [23]:
has_reviews[has_reviews['has_human_closing_user'] == False]

,id,number,user,user_id,agent,repo_url,html_url,reviews_url,reviews_data,review_counts_map,closed_by_user,closing_method,has_human_closing_user
326,2303153623,15991,renovate[bot],29139614,Human,https://api.github.com/repos/grafana/loki,https://github.com/grafana/loki/pull/15991,https://api.github.com/repos/grafana/loki/pull...,"[{'user': 'renovate-approve[bot]', 'state': 'A...",{'renovate-approve[bot]': 1},renovate[bot],Merged,False
743,2437889808,4445,pablonyx,171597620,Human,https://api.github.com/repos/onyx-dot-app/onyx,https://github.com/onyx-dot-app/onyx/pull/4445,https://api.github.com/repos/onyx-dot-app/onyx...,"[{'user': 'greptile-apps[bot]', 'state': 'COMM...",{'greptile-apps[bot]': 1},github-actions[bot],Closed,False
1532,2424036899,4038,boxmoji,22225883,Human,https://api.github.com/repos/box/box-ui-elements,https://github.com/box/box-ui-elements/pull/4038,https://api.github.com/repos/box/box-ui-elemen...,"[{'user': 'greg-in-a-box', 'state': 'APPROVED'...",{'greg-in-a-box': 1},mergify[bot],Merged,False
1533,2364985069,3998,boxmoji,22225883,Human,https://api.github.com/repos/box/box-ui-elements,https://github.com/box/box-ui-elements/pull/3998,https://api.github.com/repos/box/box-ui-elemen...,"[{'user': 'greg-in-a-box', 'state': 'APPROVED'...",{'greg-in-a-box': 1},mergify[bot],Merged,False
1534,2486762465,4079,boxmoji,22225883,Human,https://api.github.com/repos/box/box-ui-elements,https://github.com/box/box-ui-elements/pull/4079,https://api.github.com/repos/box/box-ui-elemen...,"[{'user': 'greg-in-a-box', 'state': 'APPROVED'...",{'greg-in-a-box': 1},mergify[bot],Merged,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8889,2903765641,355,devin-ai-integration[bot],158243242,Devin,https://api.github.com/repos/different-ai/note...,https://github.com/different-ai/note-companion...,https://api.github.com/repos/different-ai/note...,"[{'user': 'greptile-apps[bot]', 'state': 'COMM...",{'greptile-apps[bot]': 1},devin-ai-integration[bot],Closed,False
8931,2812399095,38849,devin-ai-integration[bot],158243242,Devin,https://api.github.com/repos/appsmithorg/appsmith,https://github.com/appsmithorg/appsmith/pull/3...,https://api.github.com/repos/appsmithorg/appsm...,"[{'user': 'KelvinOm', 'state': 'COMMENTED', 'b...",{'KelvinOm': 1},devin-ai-integration[bot],Closed,False
8934,2814227883,52583,devin-ai-integration[bot],158243242,Devin,https://api.github.com/repos/airbytehq/airbyte,https://github.com/airbytehq/airbyte/pull/52583,https://api.github.com/repos/airbytehq/airbyte...,"[{'user': 'aaronsteers', 'state': 'COMMENTED',...",{'aaronsteers': 1},devin-ai-integration[bot],Closed,False
8976,2950313797,377,devin-ai-integration[bot],158243242,Devin,https://api.github.com/repos/different-ai/note...,https://github.com/different-ai/note-companion...,https://api.github.com/repos/different-ai/note...,"[{'user': 'greptile-apps[bot]', 'state': 'COMM...",{'greptile-apps[bot]': 1},devin-ai-integration[bot],Closed,False


In [24]:
# Se quiser salvar:
has_reviews.to_parquet(r'output_files\prs_reviews.parquet', index=False)